https://www.kaggle.com/tunguz/atomic-distances-with-h2o-automl

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import GroupKFold

import os
print(os.listdir("../input"))

['sample_submission.csv', 'scalar_coupling_contributions.csv', 'structures', 'test.csv', 'structures.csv', 'dipole_moments.csv', 'mulliken_charges.csv', 'train.csv', 'magnetic_shielding_tensors.csv', 'potential_energy.csv']


In [5]:
train = pd.read_csv('../input/train.csv', index_col='id')
test = pd.read_csv('../input/test.csv', index_col='id')

/home/nagae/anaconda3/envs/gpu-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
train.head()

,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [7]:
structures = pd.read_csv('../input/structures.csv')
display(structures.head())

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [8]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

In [9]:
train.head()

,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [10]:
%%time
# This block is SPPED UP

train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
test_p_0 = test[['x_0', 'y_0', 'z_0']].values
test_p_1 = test[['x_1', 'y_1', 'z_1']].values

train['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test['dist'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)

CPU times: user 492 ms, sys: 224 ms, total: 716 ms
Wall time: 218 ms


In [11]:
molecules = train.pop('molecule_name')
test = test.drop('molecule_name', axis=1)

In [12]:
train.head()

,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist
0,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953
1,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120
2,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147
3,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157
4,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,1.091952


In [13]:
train['fold']=0

In [14]:
n_splits = 3
gkf = GroupKFold(n_splits=n_splits) # we're going to split folds by molecules


for fold, (in_index, oof_index) in enumerate(gkf.split(train, groups=molecules)):
    train.loc[oof_index, 'fold'] = fold

In [19]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init()

3.18.0.2
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /home/nagae/anaconda3/envs/gpu-env/h2o_jar/h2o.jar
  Ice root: /tmp/tmp2i9ze4rm
  JVM stdout: /tmp/tmp2i9ze4rm/h2o_nagae_started_from_python.out
  JVM stderr: /tmp/tmp2i9ze4rm/h2o_nagae_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Tokyo
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"1 year, 5 months and 7 days !!!"
H2O cluster name:,H2O_from_python_nagae_7ypy0p
H2O cluster total nodes:,1
H2O cluster free memory:,26.67 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [20]:
train = h2o.H2OFrame(train)

/home/nagae/anaconda3/envs/gpu-env/lib/python3.6/site-packages/h2o/utils/shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
train['type'] = train['type'].asfactor()
train['atom_0'] = train['atom_0'].asfactor()
train['atom_1'] = train['atom_1'].asfactor()

test['type'] = test['type'].asfactor()
test['atom_0'] = test['atom_0'].asfactor()
test['atom_1'] = test['atom_1'].asfactor()

In [23]:
x = test.columns
y = 'scalar_coupling_constant'

In [24]:
aml = H2OAutoML(max_models=2, seed=47, max_runtime_secs=3600)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [25]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mae,rmsle
DRF_0_AutoML_20190812_214400,10.2566,3.2026,1.96633,nan
StackedEnsemble_BestOfFamily_0_AutoML_20190812_214400,10.4153,3.22727,1.99325,nan
StackedEnsemble_AllModels_0_AutoML_20190812_214400,10.4153,3.22727,1.99325,nan
XRT_0_AutoML_20190812_214400,10.7081,3.27233,2.03139,nan


In [26]:
aml.leader

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_0_AutoML_20190812_214400


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 10.299018681750631
RMSE: 3.209208419805518
MAE: 1.9677778772489403
RMSLE: NaN
Mean Residual Deviance: 10.299018681750631

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 10.065618374733699
RMSE: 3.1726358717529655
MAE: 1.9485228638357468
RMSLE: NaN
Mean Residual Deviance: 10.065618374733699

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 10.256647890535174
RMSE: 3.2026001765027075
MAE: 1.9663338594358073
RMSLE: NaN
Mean Residual Deviance: 10.256647890535174
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.9663339,0.0026694,1.964975,1.9713178,1.9638929,1.961388,1.9700956
mean_residual_deviance,10.256648,0.0480623,10.209623,10.3421335,10.207376,10.187573,10.3365345
mse,10.256648,0.0480623,10.209623,10.3421335,10.207376,10.187573,10.3365345
r2,0.9916002,0.0000336,0.9916168,0.9915481,0.9916148,0.9916736,0.991548
residual_deviance,10.256648,0.0480623,10.209623,10.3421335,10.207376,10.187573,10.3365345
rmse,3.2025826,0.0074991,3.19525,3.2159188,3.1948984,3.191798,3.215048
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2019-08-12 22:10:34,26 min 32.579 sec,0.0,nan,nan,nan,nan,nan,nan
,2019-08-12 22:10:39,26 min 37.530 sec,1.0,3.7899192,2.2197120,14.3634876,3.7852763,2.2130411,14.3283169
,2019-08-12 22:10:44,26 min 42.461 sec,2.0,3.7332233,2.1991541,13.9369561,3.5021824,2.1084491,12.2652815
,2019-08-12 22:10:50,26 min 47.939 sec,3.0,3.6872095,2.1697406,13.5955142,3.3951238,2.0565290,11.5268658
,2019-08-12 22:10:56,26 min 53.828 sec,4.0,3.6178753,2.1369106,13.0890213,3.3294574,2.0224430,11.0852868
,2019-08-12 22:11:04,27 min 1.866 sec,5.0,3.5636774,2.1141940,12.6997964,3.2956240,2.0043219,10.8611373
,2019-08-12 22:11:15,27 min 12.927 sec,7.0,3.4838557,2.0772038,12.1372504,3.2582818,1.9825086,10.6164002
,2019-08-12 22:11:26,27 min 23.837 sec,9.0,3.4149710,2.0522642,11.6620272,3.2354211,1.9739221,10.4679499
,2019-08-12 22:11:43,27 min 40.859 sec,12.0,3.3524621,2.0268720,11.2390022,3.2180459,1.9658370,10.3558194
,2019-08-12 22:12:00,27 min 57.950 sec,15.0,3.3103402,2.0082076,10.9583523,3.2019920,1.9576451,10.2527525


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
dist,86832185344.0000000,1.0,0.4868932
type,80921952256.0000000,0.9319350,0.4537529
atom_index_1,3612665600.0000000,0.0416051,0.0202573
atom_1,3550153216.0000000,0.0408852,0.0199067
y_1,1022671616.0000000,0.0117776,0.0057344
y_0,848010624.0000000,0.0097661,0.0047550
atom_index_0,695163136.0000000,0.0080058,0.0038980
x_0,301652352.0000000,0.0034740,0.0016915
z_0,237515648.0000000,0.0027353,0.0013318
x_1,183995888.0000000,0.0021190,0.0010317


In [27]:
preds = aml.predict(test)
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission['scalar_coupling_constant'] = preds.as_data_frame().values.flatten()
sample_submission.to_csv('h2o_submission_3.csv', index=False)

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
